# Optimizing Agent Behavior in Dynamic Environments using Q-learning and ε-greedy Exploration

## Machine Learning Course - Reinforcement Learning Project 

## NCSR Demokritos 2022-2023

### Authors: Alexandros Filios - mtn2219 & mtn2221 - Chiotis Nikolaos

## 1 Introduction

In this report, we will present an implementation of a Q-learning, ε-greedy agent system for a repeated coordination game. The game is defined as a matrix with row column indexes called 1 and 2, where the payoff for each action combination is given. The agents belong to one of two types, X and Y, with X preferring action 1 over action 2 and Y preferring action 2 over action 1. The agents communicate with each other sparsely in an adjacency graph of our choice, and have no knowledge of the game or the repertoire of actions of the other agents.

The Q-learning, ε-greedy algorithm is a popular method for solving multi-agent reinforcement learning problems, where the agents learn to make optimal decisions based on their own actions and the rewards received from the environment. The ε-greedy exploration strategy allows the agents to balance exploitation of their current knowledge with exploration of new actions.

Our implementation consists of running a specified number of episodes, with the ε value starting at 1 and decreasing by 0.01 every X episodes. At the beginning of each episode, the time is set to 0, and each agent chooses an action using the ε-greedy strategy. After all agents have performed their actions, the environment announces the rewards to each agent, and the Q-values are updated using the Q-learning update rule. We track the energy value and the average total reward for each agent at each point in time, and create charts to visualize this data.

The main goal of this report is to explore whether the agents converge to an equilibrium and how the "equilibrium" between X and Y agents affects their convergence. We will also include the code used, charts showing the energy value per episode, and charts for each agent type showing their average total reward at each point in time.

## 2. Theoretical background & Methodology

### 2.1 Q-Learning

Q-learning is a type of reinforcement learning algorithm that is used to find the optimal action-selection policy for an agent in a specific environment. The basic idea behind Q-learning is to use a function, called the Q-function, to estimate the expected long-term reward for an agent when it takes a certain action in a given state. The Q-function is typically represented by a table or a neural network, and it is updated during the learning process to more accurately reflect the expected rewards for different actions in different states.

The Q-function is defined as $Q(s,a)$, where $s$ is the current state, and $a$ is the action taken in that state. The Q-function estimates the expected cumulative future reward for an agent starting from state %s% and taking action $a$. The agent's goal is to find the action-value function $Q^*(s,a)$ that maximizes the expected cumulative reward.

In Q-learning, the agent interacts with the environment by taking actions and receiving rewards. At each time step, the agent selects an action based on its current state and the current estimates of the Q-function. The agent then receives a reward for the action and updates the Q-function to more accurately reflect the new information about the expected rewards for that action in that state. This process is repeated until the agent reaches a stopping criterion, such as a maximum number of iterations or a satisfactory level of performance.

The Q-function is updated using the Q-learning update rule, which is a variation of the Bellman equation, a fundamental equation in dynamic programming. The Q-learning update rule states that the Q-value of a state-action pair is updated by taking the current Q-value and adjusting it by a small value proportional to the difference between the expected reward and the current estimate. 

The update rule is as follows:

$$ Q(s,a) = Q(s,a) + α(r + γ maxQ(s',a') - Q(s,a)) $$

Where:

- $s$: represents the current state of an agent in an environment.
- $a$: current action taken by the agent when it is in a particular state.
- $α$: learning rate
- $r$: reward for taking action a in state s the agent will get a positive or negative reward).
- $γ$: discount factor
- $s'$: next state
- $a'$: next action
- $maxQ(s',a')$: maximum Q-value over all actions in next state s'

The Q-function is used to select actions by choosing the action that has the highest Q-value for the current state. This is known as the greedy policy.

### 2.2 ε-Greedy algorithm

The ε-greedy algorithm is a method used to balance exploration and exploitation during the learning process. In the ε-greedy algorithm, the agent selects actions according to the following rule: with probability ε, a random action is chosen, and with probability 1-ε, the action that has the highest estimated Q-value is chosen. The value of ε is called the exploration rate and it's used to control the trade-off between exploration and exploitation. At the beginning of the learning process, ε is set to a high value to allow the agent to explore the environment and gather information about the different states and actions. As the learning process progresses, ε is gradually decreased, allowing the agent to become more exploitation-focused and make use of the information it has gathered.

In summary, Q-learning is a model-free, off-policy algorithm for learning control policies, where the Q-function is used to approximate the optimal action-value function. It uses the Q-learning update rule which uses the Bellman equation and the agent interacts with the environment and learns by trial and error

## 3. Implementation & Experinments

In [57]:
import numpy as np
import math
# Define the game matrix
game_matrix = np.array([[1.1, 0.0], [0.0, 1.1]])

# Define the number of agents and the types of agents
num_agents = 7

# Define the types of agents
agent_types = ["X", "Y"]

# Initialize the Q-values for each agent and each state-action pair to a small random value
q_values = {agent_type: {i: np.random.rand(2) * 0.1 for i in range(num_agents)} for agent_type in agent_types}

# Define the adjacency graph
# For example, adjacency_graph[0][1] = True means agent 0 is connected to agent 1
adjacency_graph = np.random.randint(0, 2, size=(num_agents, num_agents))

# Define the number of episodes and the episode length
num_episodes = 2500 #TODO: make it 2500
episode_length = 50

# Define the discount factor
gamma = 0.9

# Define the initial exploration probability and the number of episodes over which it decreases
epsilon = 100 # 100/100
epsilon_decay_episodes = 20

# Initialize the rewards for each agent
rewards = {agent_type: np.zeros(num_agents) for agent_type in agent_types}

# The matrix we are going to keep the data in
result = []

# Initialize the action value
action_value = 0

# Initialize the episode counter
episode_counter = 0

# Run the episodes
for episode in range(num_episodes):
    # Update the exploration probability
    if episode_counter % epsilon_decay_episodes == 0 and episode_counter != 0 and episode_counter <= 2000:
        epsilon -= 1
    
    # Initialize the episode rewards
    episode_rewards = {agent_type: np.zeros(num_agents) for agent_type in agent_types}
    
    # Run the episode steps
    for step in range(episode_length):
        
        # Initialize the actions for each agent
        actions = {agent_type: np.zeros(num_agents) for agent_type in agent_types}
    
        # Get the actions for each agent
        for i in range(num_agents):
            agent_type = agent_types[i % 2]
            if np.random.rand() < epsilon/100:
                # Choose a random action with probability epsilon
                actions[agent_type][i] = np.random.randint(2)
            else:
                # Choose the action with the highest Q-value with probability 1-epsilon
                actions[agent_type][i] = np.argmax(q_values[agent_type][i])
        
        # Compute the rewards for each agent
        for i in range(num_agents):
            agent_type = agent_types[i % 2]
            agent_rewards = np.zeros(num_agents)
            for j in range(num_agents):
                if adjacency_graph[i][j]:
                    agent_rewards[j] = game_matrix[int(actions[agent_type][i]), int(actions[agent_types[j % 2]][j])]
            rewards[agent_type][i] = np.mean(agent_rewards)
            
        # Update the Q-values for each agent
        for i in range(num_agents):
            agent_type = agent_types[i % 2]
            q_values[agent_type][i][int(actions[agent_type][i])] = rewards[agent_type][i] + gamma * np.max(q_values[agent_type][i])

        # Update the episode rewards for each agent
        for agent_type in agent_types:
            episode_rewards[agent_type] += rewards[agent_type]
            #print(episode_rewards)
        
    # Update the action value
    for agent_type in agent_types:        
        action_value_X = np.mean(np.sum(episode_rewards[agent_type], axis=0))
        

    # Update the episode counter
    episode_counter += 1
    
    print(energy_value)

123.98571428571431
242.6285714285715
361.42857142857144
481.64285714285717
601.7
727.2571428571429
842.1285714285715
971.4571428571429
1094.5
1218.4857142857143
1335.4
1457.3428571428574
1578.0285714285717
1698.5571428571432
1820.1857142857148
1940.7142857142862
2065.1714285714293
2185.7000000000007
2312.5142857142864
2432.728571428572
2552.000000000001
2672.528571428572
2792.428571428572
2917.985714285715
3040.5571428571434
3159.828571428572
3282.7142857142867
3404.8142857142866
3527.700000000001
3647.914285714287
3769.07142857143
3888.6571428571438
4014.842857142858
4136.785714285716
4255.9000000000015
4381.300000000001
4503.8714285714295
4626.442857142858
4744.3
4864.9857142857145
4988.814285714286
5106.9857142857145
5228.614285714286
5350.871428571429
5469.671428571429
5595.071428571429
5713.242857142857
5832.9857142857145
5954.614285714286
6071.528571428571
6192.842857142857
6314.628571428571
6437.357142857143
6559.9285714285725
6678.571428571429
6799.100000000001
6921.67142857143

62531.54285714287
62651.60000000001
62772.28571428573
62895.32857142859
63021.82857142859
63142.35714285716
63264.6142857143
63383.257142857154
63513.371428571445
63635.94285714287
63762.128571428584
63887.371428571445
64013.871428571445
64138.32857142859
64263.72857142858
64391.32857142858
64518.45714285715
64635.685714285726
64755.58571428572
64874.228571428575
65002.614285714284
65126.91428571428
65253.41428571428
65380.85714285714
65505.31428571428
65631.1857142857
65747.62857142856
65873.81428571427
65998.89999999998
66126.49999999999
66253.15714285713
66376.9857142857
66499.39999999998
66628.25714285712
66750.51428571428
66875.44285714286
67006.65714285715
67134.72857142857
67263.27142857143
67388.35714285714
67510.14285714286
67634.59999999999
67759.05714285714
67887.6
68010.8
68133.37142857142
68267.09999999999
68393.91428571427
68518.6857142857
68637.95714285714
68756.28571428571
68879.64285714286
69007.55714285714
69133.58571428571
69262.6
69388.94285714287
69514.18571428572


128933.51428571425
129070.22857142854
129214.1714285714
129341.77142857139
129480.52857142853
129620.85714285709
129762.91428571423
129899.15714285708
130036.81428571422
130182.6428571428
130320.45714285708
130457.17142857137
130599.38571428564
130729.18571428565
130865.58571428565
131013.61428571423
131162.27142857137
131297.88571428566
131427.8428571428
131567.38571428566
131718.39999999994
131855.42857142852
131996.5428571428
132135.1428571428
132270.59999999995
132408.7285714285
132551.7285714285
132687.18571428565
132815.4142857142
132951.49999999994
133081.92857142852
133224.45714285708
133359.4428571428
133491.75714285707
133640.88571428563
133777.28571428562
133912.27142857134
134049.14285714278
134192.29999999993
134327.7571428571
134467.61428571425
134604.4857142857
134746.85714285713
134887.34285714282
135026.25714285712
135172.24285714285
135302.82857142857
135441.90000000002
135579.08571428573
135709.67142857146
135848.4285714286
135997.71428571432
136129.24285714288
13626

203958.8571428571
204133.4428571428
204290.27142857137
204464.69999999995
204635.0428571428
204802.24285714282
204963.1571428571
205118.09999999995
205283.88571428566
205450.61428571423
205619.85714285707
205792.39999999994
205968.55714285705
206126.7999999999
206289.12857142847
206457.5857142856
206621.64285714275
206786.64285714275
206954.15714285703
207116.32857142846
207278.81428571418
207450.0999999999
207603.15714285703
207772.24285714273
207944.157142857
208094.38571428557
208266.29999999987
208435.54285714272
208596.7714285713
208765.38571428557
208945.4714285713
209102.7714285713
209274.3714285713
209450.3714285713
209630.14285714272
209789.01428571413
209958.09999999983
210120.4285714284
210293.91428571413
210451.84285714268
210625.0142857141
210794.4142857141
210975.75714285695
211146.57142857125
211320.68571428553
211485.52857142838
211664.5142857141
211826.21428571412
212002.84285714268
212170.8285714284
212352.01428571413
212511.0428571427
212679.97142857124
212843.085714

289453.21428571414
289656.39999999985
289859.42857142846
290052.55714285705
290254.48571428563
290458.7714285713
290663.057142857
290867.3428571427
291071.62857142836
291275.91428571404
291480.1999999997
291684.4857142854
291888.7714285711
292093.05714285676
292297.34285714244
292501.6285714281
292705.9142857138
292910.1999999995
293114.48571428517
293318.77142857085
293523.05714285653
293727.3428571422
293931.6285714279
294135.9142857136
294340.19999999925
294544.48571428494
294748.7714285706
294953.0571428563
295157.342857142
295361.62857142766
295565.91428571334
295770.199999999
295974.4857142847
296178.7714285704
296383.05714285607
296587.34285714175
296791.6285714274
296995.9142857131
297200.1999999988
297404.48571428447
297608.77142857015
297813.05714285583
298017.3428571415
298221.6285714272
298425.9142857129
298630.19999999856
298834.48571428424
299038.7714285699
299243.0571428556
299447.3428571413
299651.62857142696
299855.91428571264
300060.1999999983
300264.485714284
300468.

386677.3428571271
386881.62857141276
387085.91428569844
387290.1999999841
387494.4857142698
387698.7714285555
387903.05714284116
388107.34285712685
388311.6285714125
388515.9142856982
388720.1999999839
388924.48571426957
389128.77142855525
389333.05714284093
389537.3428571266
389741.6285714123
389945.914285698
390150.19999998366
390354.48571426934
390558.771428555
390763.0571428407
390967.3428571264
391171.62857141206
391375.91428569774
391580.1999999834
391784.4857142691
391988.7714285548
392193.05714284047
392397.34285712615


In [17]:
# Plot the energy value and the average total rewards for each agent type
# Code for plotting goes here

## 4. Epilogue